In [131]:
# import pydicom library
import pydicom

# import matplotlib and numpy
import matplotlib.pyplot as plt 
import matplotlib.image as mpimage
import matplotlib.patches as patches

from matplotlib import cm
import colorcet as cc
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1 import make_axes_locatable

import numpy as np 
import pandas as pd
import cv2

# import operating system and glob libraries
import os

import time
from datetime import datetime

# import some useful date functions
from datetime import datetime

import scipy
from scipy import ndimage

# this following line tells Jupyter to display images here in the browser, 
# rather than in separate window.
%matplotlib inline 

In [132]:
%%capture
%run numpngw.ipynb

In [133]:
dicomImage_issues = []
observationImage_issues = []

#CBIS_DDSM_dir   = "/Users/kasparlund/AICodeData/x-ray/CBIS-DDSM-mammography/"
CBIS_DDSM_dir   = "/Volumes/LaCie/CBIS-DDSM-mammography/"
CBIS_DDSM_Dicom = CBIS_DDSM_dir+"official-Dicom-files/"
CBIS_DDSM_cvs   = CBIS_DDSM_dir+"official-csv-files/"
CBIS_DDSM_describtion_of_calcification_trainingdata = CBIS_DDSM_cvs + "calc_case_description_train_set.csv"
CBIS_DDSM_describtion_of_calcification_testdata     = CBIS_DDSM_cvs + "calc_case_description_test_set.csv"
CBIS_DDSM_describtion_of_mass_trainingdata          = CBIS_DDSM_cvs + "mass_case_description_train_set.csv"
CBIS_DDSM_describtion_of_mass_testdata              = CBIS_DDSM_cvs + "mass_case_description_test_set.csv"

CBIS_DDSM_Converted = "/Users/kasparlund/AICodeData/x-ray/CBIS-DDSM-mammography/converted_images/"
os.makedirs(CBIS_DDSM_Converted,   exist_ok=True)


## 2 Examining data description files

In [134]:
calcification_trainingdata = pd.read_csv(CBIS_DDSM_describtion_of_calcification_trainingdata) 
mass_trainingdata          = pd.read_csv(CBIS_DDSM_describtion_of_mass_trainingdata) 

calcification_testdata = pd.read_csv(CBIS_DDSM_describtion_of_calcification_testdata) 
mass_testdata          = pd.read_csv(CBIS_DDSM_describtion_of_mass_testdata) 

#prepare merge of all datasets
if len(calcification_trainingdata.columns) != len(mass_trainingdata.columns) :
    print("len(calcification_trainingdata.columns) != len(mass_trainingdata.columns)")
if len(calcification_trainingdata.columns) != len(calcification_testdata.columns) :
    print("len(calcification_trainingdata.columns) != len(calcification_testdata.columns)")
if len(calcification_trainingdata.columns) != len(mass_testdata.columns) :
    print("len(calcification_trainingdata.columns) != len(mass_testdata.columns)")

mass_trainingdata      = mass_trainingdata.reindex(      calcification_trainingdata.columns, axis="columns" )
calcification_testdata = calcification_testdata.reindex( calcification_trainingdata.columns, axis="columns" )
mass_testdata          = mass_testdata.reindex(          calcification_trainingdata.columns, axis="columns" )

#add column with purpose
calcification_trainingdata.insert(1, "purpose", "train" )
mass_trainingdata.insert(1, "purpose", "train" )
calcification_testdata.insert(1, "purpose", "test" )
mass_testdata.insert(1, "purpose", "test" )

#normalize the column names by creating composede names for the 2 columns i the calcification and mass datasets 
#used to characterize form and shape of observations:
# "calc type - mass_shape"
# "calc distribution - mass margins"
calcification_trainingdata.columns = ['patient_id', 'purpose', 'breast density', 'left or right breast', 'image view',
       'abnormality id', 'abnormality type', 'calc type - mass_shape',
       'calc distribution - mass margins', 'assessment', 'pathology', 'subtlety',
       'image file path', 'cropped image file path', 'ROI mask file path']
mass_trainingdata.columns = calcification_trainingdata.columns
mass_testdata.columns = calcification_testdata.columns = calcification_trainingdata.columns

#concatenate the datasets so we only have a dataset for training and one for test
allData = pd.concat( [calcification_trainingdata, mass_trainingdata, calcification_testdata, mass_testdata] )

#clean som irregularities we have meet
removeEOF = lambda x: x.replace('\n', '').replace('\r', '')
allData.loc[:,"image file path"]         = allData.loc[:,"image file path"].apply( removeEOF )
allData.loc[:,"cropped image file path"] = allData.loc[:,"cropped image file path"].apply( removeEOF )
allData.loc[:,"ROI mask file path"]      = allData.loc[:,"ROI mask file path"].apply( removeEOF) 

#extract case_id from the first local dir in "image file path". 
#We could have used the existing case_id but perfer to calculate it from the image paths 
case_id = allData.loc[:,"image file path"].str.extract(r"^(?P<case_id>[^\/]+)\/")
allData.insert(0, "case_id", case_id.case_id )

#The obs_id for "cropped image file path" and "ROI mask file path" is the same 
obs_id = allData.loc[:,"cropped image file path"].str.extract(r"^(?P<obs_id>[^\/]+)\/")
allData.insert(1, "obs_id", obs_id.obs_id )

allData["counter"] = np.ones( [allData.shape[0],1], dtype=np.uint8 )
allData.fillna(0, inplace=True)

In [135]:
#the dicom image cannot be read correctly for this image so remove it
ix_remove = False == np.isin( allData.case_id, 
                             ["Calc-Training_P_01264_RIGHT_MLO", 
                              "Mass-Training_P_00225_RIGHT_CC"] )
#burde måske fjerne flg fordi billedene med observationernes masker var mindre end dicombilledet
#Mass-Training_P_02092_LEFT_MLO_1 
#Mass-Training_P_02092_LEFT_CC_1

allData = allData[ix_remove]

# identify and remove cases with more than 1 type of observation in the image
p  = pd.pivot_table(allData, values="counter", index=["case_id"], 
                    columns= "pathology", 
#                    columns= "subtlety", 
                    aggfunc=np.mean, fill_value=0, 
#                    margins = True, margins_name= 'Total count' 
                   )
p["different_cases"] = p.BENIGN.add( p.BENIGN_WITHOUT_CALLBACK,fill_value=0).add( p.MALIGNANT, fill_value=0)
p.sort_values("different_cases",ascending=True)
ix_more_different_cases = p.different_cases>1

ix_remove = np.isin( allData.case_id, p[ix_more_different_cases].index )


In [136]:
#allData = allData[ix_remove==False]

"""
Index([
'Calc-Test_P_00353_LEFT_CC', 
'Calc-Test_P_00353_LEFT_MLO',
'Calc-Training_P_00600_LEFT_CC', 
'Calc-Training_P_00600_LEFT_MLO',
'Calc-Training_P_00858_RIGHT_CC', 
'Calc-Training_P_00858_RIGHT_MLO',
'Calc-Training_P_00937_RIGHT_CC', 
'Calc-Training_P_00937_RIGHT_MLO',
'Calc-Training_P_01284_RIGHT_MLO', 
'Calc-Training_P_01819_LEFT_CC',
'Calc-Training_P_01819_LEFT_MLO', 
'Mass-Test_P_00969_LEFT_CC',
'Mass-Test_P_00969_LEFT_MLO', 
'Mass-Training_P_00419_LEFT_CC',
'Mass-Training_P_00419_LEFT_MLO', 
'Mass-Training_P_00797_LEFT_CC',
'Mass-Training_P_00797_LEFT_MLO', 
'Mass-Training_P_01103_RIGHT_CC',
'Mass-Training_P_01103_RIGHT_MLO']
"""

In [137]:
# Simplify classification by changing BENIGN_WITHOUT_CALLBACK into BENIGN
simple_pathology = allData.pathology.copy()
simple_pathology[ simple_pathology == "BENIGN_WITHOUT_CALLBACK" ] = "BENIGN"
allData.insert( 2, "simple_pathology", simple_pathology )

In [138]:
os.makedirs(CBIS_DDSM_Converted,   exist_ok=True)
allData.to_csv(CBIS_DDSM_Converted + "/converted_data.csv", sep=";")

uniqueCases = np.unique( allData.case_id )
uniqueObservations = np.unique( allData.obs_id )
print("number of unique cases: ", len(uniquecases), " uniqueObservations: ", len(uniqueObservations) )

pd.pivot_table( allData, values="counter", index=["case_id"], 
                columns= "simple_pathology", 
                aggfunc=np.sum, fill_value=0, 
                margins = True, margins_name= 'Total count' 
              )

The redundant image path in test and training is due to multiple ROI pr image.
For the time being we are only interested in training with the images so we will ignore the masks

## 1. Reading and viewing a DICOM image

### Here follows the image processing

In [139]:
def plotDicomWithSeedpoint(img, title,   seedpoint, angle, width, radius, colormap ):

    height = round( (width*img.shape[0]) /img.shape[1] )
    fig = plt.figure(figsize = (width,height)) 
    plt.suptitle(title, fontsize=12)
    plt.tight_layout()
    ax = fig.subplots()
    ax.imshow(img, cmap=colormap)
    
    circ = patches.Circle(seedpoint, radius, alpha=0.8, fc='yellow')
    ax.add_patch(circ)
    
    
    x  = seedpoint[0]
    y  = seedpoint[1]
    
    rv  = min( y, img.shape[1]-y )*0.5
    dx = rv * np.cos(angle)
    dy = rv * np.sin(angle)
    ax.plot([x - dx, x, x + dx], [y - dy, y, y + dy], 'ro-' )
    
    rh  = min( x, img.shape[0]-x )*0.5
    dx = rh * np.cos(angle + np.radians(90))
    dy = rh * np.sin(angle + np.radians(90))
    ax.plot([x - dx, x, x + dx], [y - dy, y, y + dy], 'wo-' )
    
    plt.show()

In [140]:
#plot histograme
def plotHistogramsBeforeAndAfterMasking(dicomImage, cleanDicomImage, title):
    fig=plt.figure(figsize = (12,12), dpi=100) 
    plt.suptitle(title, fontsize=15)
    plt.tight_layout()


    nb_plot_bins = 512
    ax1 = fig.add_subplot(221)
    ax1.set_title("Original dicom Image")
    plt.hist(dicomImage.flatten(), nb_plot_bins, [0.01, 0.99] )

    ax2 = fig.add_subplot(222)
    ax2.set_title("Clean dicoemImage: " )
    plt.hist(cleanDicomImage.flatten(), nb_plot_bins, [0.01, 0.99] )
    plt.show()

In [141]:
def plotMaskProfile( profile_before, profile_after, diffIdx_before, diffIdx_after ):
    fig = plt.figure(figsize=(8,8) )
    plt.title("sum of floodmask columns")
    plt.plot(profile_after, color = "red")
    plt.plot(profile_before, color="black")
    plt.plot(diffIdx_before, color="blue")
    plt.plot(diffIdx_after, color="green")

In [142]:
def DoublePlot(mainTitle, img1, title1, img2, title2, colormap, width = 16 ):

    height = round( (width*img1.shape[0]) /img1.shape[1] )

    fig = plt.figure(figsize = (width,height)) 
    plt.tight_layout()
    #fig.suptitle(mainTitle)

    ax1 = fig.add_subplot(221)
    ax1.set_title(title1)
    im1 = ax1.imshow(img1, cmap=colormap)
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes("bottom", "5%", pad="3%")
    colorbar = fig.colorbar(im1, cax=cax, orientation="horizontal")

    ax2 = fig.add_subplot(222)
    ax2.set_title(title2)

    im2 = ax2.imshow(img2, cmap=colormap)
    divider2 = make_axes_locatable(ax2)
    cax2 = divider2.append_axes("bottom", "5%", pad="3%")
    colorbar2 = fig.colorbar(im2, cax=cax2, orientation="horizontal")


    plt.show()    
    return

In [143]:
def simplePlot(img, title, cmap):
    fig = plt.figure(figsize=(8,8) )
    plt.title(title)
    plt.imshow(img, cmap)

In [144]:
# find center of mammography region to seed the floodfill
def calcMasscenterOrientation(image):
    moments = cv2.moments(image)
    #print("moments: \n", str( moments) )
    m00 = moments["m00"]
    m10 = moments["m10"]
    m01 = moments["m01"]
    m11 = moments["m11"]
    m20 = moments["m20"]
    m02 = moments["m02"]
    
    x =  y = angle = 0
    if m00 > 0 :
        x =  m10 / m00
        y =  m01 / m00    
        u11 = (m11 - x * m01) / m00
        u20 = (m20 - x * m10) / m00
        u02 = (m02 - y * m01) / m00
        angle = 0.5 * np.arctan(2.0 * u11 / (u20 - u02))
    else: 
        dicomImage_issues.append( ("?", "calcMasscenterOrientation m00=0") )
    return x, y, angle

In [145]:
# Remove the image border and corners that are dominated by artefacts from the scanning og the images.

def MyPolyLine( img, polyline, thickness,  removeMaskcolor = 0 ):
    for i in range(1,len(polyline)):
        cv2.line( img, polyline[i-1], polyline[i], removeMaskcolor, thickness)
    cv2.line( img, polyline[i], polyline[0], removeMaskcolor, thickness)
    return 

def MoveShape( points, movement ):
    newShape = points.copy()
    for i in range(0,len(newShape)) :
        newShape[i] += movement
    return newShape

def removeBorder( imageMask, inset = 25, corner_width=300, removeMaskcolor = 0 ):
    imageMask = imageMask.copy()
    maskSize = imageMask.shape
    
    boxline = []
    boxline.append((inset,             inset))
    boxline.append((maskSize[1]-inset, inset))
    boxline.append((maskSize[1]-inset, maskSize[0]-inset))
    boxline.append((inset,             maskSize[0]-inset))
    
    MyPolyLine(imageMask, boxline, inset*2+1, removeMaskcolor)    

    #mask topledft and bottomright
    triangle_topLeftBottomRight  = np.array([ [0,0], [1,0], [0,1] ], np.int32)
    triangle_topLeftBottomRight *= corner_width
    cv2.fillConvexPoly(imageMask, triangle_topLeftBottomRight, removeMaskcolor)

    movementBottomRight = [maskSize[1], maskSize[0]]
    triangle_topLeftBottomRight = MoveShape( -triangle_topLeftBottomRight, movementBottomRight )
    cv2.fillConvexPoly(imageMask, triangle_topLeftBottomRight,removeMaskcolor)
    #print(triangle_topLeftBottomRight)

    #mask bottomledft and topright
    triangle_BottomLeftTopRight  = np.array([ [0,0], [0,-1], [1,0] ], np.int32)
    triangle_BottomLeftTopRight *= corner_width
    movementBottomLeft = [0, imageMask.shape[0]]
    triangle_bottomLeft = MoveShape( triangle_BottomLeftTopRight, movementBottomLeft )
    cv2.fillConvexPoly(imageMask, triangle_bottomLeft, removeMaskcolor)

    movementTopRight = [imageMask.shape[1], 0]
    triangle_BottomLeftTopRight = MoveShape( -triangle_BottomLeftTopRight, movementTopRight )
    a = cv2.fillConvexPoly(imageMask, triangle_BottomLeftTopRight, removeMaskcolor)
        
    return imageMask

In [146]:
def makeImageWithContour( image, dicomImage_ROI ):
    img2,contours,hierarchy = cv2.findContours( dicomImage_ROI, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE )
    idx = -1
    color = (0,0,0)
    thickness = 2
    lineType = cv2.LINE_8
    return cv2.drawContours( image.copy(), contours, idx, color, thickness, cv2.LINE_8, hierarchy );

In [147]:
from scipy.interpolate import splprep, splev

def smoothMask (mask, simpleSmooth = True ):
    img2, contours, hierarchy = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    # get the max-area contour
    contours = sorted(contours, key=lambda x: cv2.contourArea(x) )[-1]
    #print("len contours:  ", len(contours) )
       
    # calc arclentgh
    arclen = cv2.arcLength(contours, True)

    # %doctest_mode approx
    eps = 0.00025
    epsilon = arclen * eps
    approx = cv2.approxPolyDP(contours, epsilon, True)
    #print("len approx:  ", len(approx) )
    #print(approx )
    #print(contours )


    # find the knot points
    if simpleSmooth :
        pts =  approx
        res_array = pts
    else:    
        pts = approx
    print("len pts:  ", len(pts) )
    
    pts = pts.reshape(pts.shape[0],2)
    pts = pd.DataFrame(pts)
    pts = pts.values.T.tolist()
    #print("length of approx: ", len(approx),  " points \n", approx )

    
    # spline parameters
    if not simpleSmooth :
        s=1.0 # smoothness parameter
        k=3 # spline order
        nest=-1 # estimate of number of knots needed (-1 = maximal)
        tck,u = splprep( pts, s=s, k=k, nest=nest )
        u_new = np.linspace(u.min(), u.max(), 90)
        x_new, y_new = splev(u_new, tck, der=0)

       # Convert it back to numpy format for opencv to be able to display it
        res_array = [[[int(i[0]), int(i[1])]] for i in zip(x_new,y_new)]
        res_array = np.asarray(res_array, dtype=np.int32)
    #print("len res_array:  ", len(res_array) )
    # print(res_array )
    
    idx = -1
    color = (0, 0, 0)
    thickness = 5
    lineType = cv2.LINE_8
    canvas = mask.copy()
    
    return cv2.fillPoly( canvas, [res_array], 1, lineType, 0)

In [148]:
def rotate(dicomImage, main_mask, dicomImage_ROI, dicomImage_cropped, minimum_angle=1.0):
    x, y, angle = calcMasscenterOrientation( main_mask )
    masscenter = (round(x),round(y))
    angle = np.degrees(angle)
    print( "masscenter from mask: ", masscenter, " angle: ", angle )
    if abs(angle) > minimum_angle : 
        dicomImage = ndimage.rotate(dicomImage,         -angle, reshape=False)
        main_mask  = ndimage.rotate(main_mask, -angle, reshape=False)
        if not dicomImage_ROI is None :
            dicomImage_ROI = ndimage.rotate(dicomImage_ROI,     -angle, reshape=False)
        if not dicomImage_cropped is None :
            dicomImage_cropped = ndimage.rotate(dicomImage_cropped, -angle)
    return masscenter, angle, dicomImage, main_mask, dicomImage_ROI, dicomImage_cropped

In [149]:
def cropRect( mask ):
    img2, contours, hierarchy = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    contours = sorted(contours, key=lambda x: cv2.contourArea(x) )[-1]
    #rect = cv2.minAreaRect(dicomImage)
    #box = cv2.boxPoints(rect)
    #box = np.int0(box)
    x,y,w,h = cv2.boundingRect(contours)
    return x,y,w,h

def crop( image, x,y,w,h ):return image[y:y+h, x:x+w]

In [150]:
def ConvertGreyScale ( img, colorArray ):
    colors = colorArray.copy()*255.0
    colors[0, :] = [0,0,0]
    colors[-1,:] = [1,1,1]
    nb_colors = colors.shape[0]
    
    x_colors  = np.linspace(0.0, 1.0, len(colors), True) 
    """ 
    print("First color: ", colors[0, :], " Last color: ", colors[-1,:], "\nFirst - last color: ", colors[-1, :] - colors[0, :], 
          "\nsum(abs(colors[-1, :] - colors[0, :])): ", sum(abs(colors[-1, :] - colors[0, :]))  )
    print("unique colors: ", np.unique(colors, axis=0).shape[0] )
    #print("unique x-colors: ", np.unique(x_colors).shape[0] )
    """
    #img = equalizedImage.astype('float64')/equalizedImage.max()
    colorImage = np.zeros( [img.shape[0],img.shape[1],3], np.uint8)
    colorImage[:,:,0] = np.interp(img, x_colors, colors[:,0] ).astype(np.uint8)
    colorImage[:,:,1] = np.interp(img, x_colors, colors[:,1] ).astype(np.uint8)
    colorImage[:,:,2] = np.interp(img, x_colors, colors[:,2] ).astype(np.uint8)
    return colorImage

In [151]:
def trimMaskRows( mask, threshhold ):
    s            = np.sum(mask,1)
    accThreshold = threshhold  * np.sum(s)
    
    #The important areas of the breast is closer to the middel of the image than the border
    #Measure how far from the middel the mask is located on a per row basis
    #The delete rows where the mask is furthest away form the important areas until 
    #we have deleted an amount of areas defined by accThreshold 
    xcs      = mask.cumsum(1)
    mid_cols = (mask.shape[1]-1)/2
    diffIdx  = np.abs( np.add(xcs.argmax(1), np.argmax(xcs>0, 1))/2 - mid_cols )
    
    #print("np.sum(s): ", np.sum(s), " accThreshold: ", accThreshold)
    
    count = 0
    i_start = 0
    i_reverse = len(s)-1
    x = []
    y = []
    acc = 0
    while acc < accThreshold:
        count += 1
        
        if diffIdx[i_start] >= diffIdx[i_reverse] :
           j = i_start
           i_start +=1
        else:
           j = i_reverse
           i_reverse -= 1
            
        x.append(j)
        y.append(s[j])
        acc += s[j]
        mask[j][:] = 0
    return mask, i_start, i_reverse

In [152]:
def readDicomImages( row ):
    image_filename         = row["image file path"]
    """
    print("image file path]:\n",        row["image file path"])
    """
    
    dImage = None
    issues = []
    try:
        dImage = pydicom.dcmread(CBIS_DDSM_Dicom + image_filename).pixel_array.astype('float32')/ 65535.0
    except:
        issues.append( (row.patient_id, "dicomImage: " + row.case_id ) )
        
    if len(issues) > 0 :
        print("issues\n", issues)
    return dImage, issues


from enum import Enum
class ObservationType(Enum):
    BENIGN=1
    MALIGNANT=2
    
def readObservationImages( row ):
    cropped_image_filename = row["cropped image file path"]
    roi_filename           = row["ROI mask file path"]
    observationType        = ObservationType.BENIGN if "BENIGN" in row["pathology"] else ObservationType.MALIGNANT
    """
    print("cropped image file path:\n", cropped_image_filename)
    print("ROI mask file path:\n]",     roi_filename)
    """
    
    dImage_ROI = dImage_observation = None
    issues = []
    
    try:
        dImage_ROI = pydicom.dcmread(CBIS_DDSM_Dicom + roi_filename)
        #print("1: dImage_ROI p.BitsAllocated " , dImage_ROI.BitsAllocated, " p.PixelRepresentation: ", 
        #      dImage_ROI.PixelRepresentation) 
    except AttributeError as e:
        issues.append( (row.patient_id, "dciomImage_ROI: "+ row.case_id ) )
        p = pydicom.dcmread(CBIS_DDSM_Dicom + cropped_image_filename, stop_before_pixels=True)
        issues.append( (row.patient_id, "1: dImage_ROI p.BitsAllocated", p.BitsAllocated) )
        issues.append( (row.patient_id, "1: dImage_observation p.PixelRepresentation: ", p.PixelRepresentation) )
        issues.append( (row.patient_id, "1: AttributeError: " + str(e)) )
        
    try:
        dImage_observation = pydicom.dcmread(CBIS_DDSM_Dicom + cropped_image_filename)
        #print("2: dImage_observation p.BitsAllocated", dImage_observation.BitsAllocated, 
        #      " p.PixelRepresentation: ", dImage_observation.PixelRepresentation) 
    except AttributeError as e:
        issues.append( (row.patient_id, "dicomImage_cropped: " + row.case_id ) )
        p = pydicom.dcmread(CBIS_DDSM_Dicom + cropped_image_filename, stop_before_pixels=True)
        issues.append( (row.patient_id, "2: dImage_observation p.BitsAllocated : ", p.BitsAllocated) )
        issues.append( (row.patient_id, "2: dImage_observation p.PixelRepresentation: ", p.PixelRepresentation) )
        issues.append( (row.patient_id, "2: AttributeError: " +  str(e)) )
         
        
    try:
        #if not dImage_observation is None: print( "\ndImage_observation.dtype: ", type(dImage_observation.pixel_array) )
        #if not dImage_observation is None: print( "dImage_ROI.dtype: ", type(dImage_ROI.pixel_array) )
            
        if not dImage_observation is None and not dImage_ROI is None:
            if (dImage_observation.Rows * dImage_observation.Columns) > (dImage_ROI.Rows * dImage_ROI.Columns):
                tmp = dImage_ROI
                dImage_ROI = dImage_observation
                dImage_observation = tmp
                
            try:
                dImage_observation = dImage_observation.pixel_array
                if   dImage_observation.dtype == np.uint8:  dImage_observation = dImage_observation.astype('float32')/255.0  
                elif dImage_observation.dtype == np.uint16: dImage_observation = dImage_observation.astype('float32')/65535.0
                #print("dImage_observation.dtype: ", dImage_observation.dtype)
            except:
                dImage_observation = None
                issues.append( (row.patient_id, "3: exception: failed to convert dImage_observation.pixel_array") )
                
            try:
                dImage_ROI = dImage_ROI.pixel_array
                
                if   dImage_ROI.dtype == np.uint8:  dImage_ROI = dImage_ROI.astype('float32')/255.0  
                elif dImage_ROI.dtype == np.uint16: dImage_ROI = dImage_ROI.astype('float32')/65535.0
                if dImage_ROI.max() > 0:    
                    dImage_ROI = ( (dImage_ROI*255.0 +0.5)/ dImage_ROI.max() ).astype(np.uint8)
                else: 
                    dImage_ROI = dImage_ROI.astype(np.uint8)
                    
                    issues.append( (row.patient_id, "4: dImage_ROI.max() == 0") )
                    p = pydicom.dcmread(CBIS_DDSM_Dicom + cropped_image_filename, stop_before_pixels=True)
                    issues.append( (row.patient_id, "4: dImage_ROI p.BitsAllocated : ", p.BitsAllocated) )
                    issues.append( (row.patient_id, "4: dImage_ROI p.PixelRepresentation: ", p.PixelRepresentation) )
                    issues.append( (row.patient_id, "4: dImage_ROI AttributeError: " + str(e) ) )
            except AttributeError as e:
                dImage_ROI = None
                issues.append( (row.patient_id, "5: exception: failed to convert dImage_ROI.pixel_array") )
                issues.append( (row.patient_id, "5: AttributeError: " + str(e)) )
        else:
            dImage_ROI = dImage_observation = None
            issues.append( (row.patient_id, "6: dImage_ROI or dImage_observation is None: " + row.case_id) )
    except:
        issues.append( (row.patient_id, "7: exception: failed to convert dImage_ROI or dImage_observation") )
      
        
    #if len(issues) > 0 : print("issues\n", issues)
    return dImage_ROI, dImage_observation, observationType, issues

In [153]:
def saveObservation( root, purpose, obs_id, observationimage, roi_mask, colorArray ):
    directory = root + "/observations/"+purpose+"/obs/"
    os.makedirs(directory, exist_ok=True)    
    main_path_filename = directory + "/" + obs_id

    if not roi_mask is None:
        cv2.imwrite(main_path_filename+"_roi.png", roi_mask)
    
    if not observationimage is None:
        imgASuint16 = (65535.0*observationimage + 0.5).astype(np.uint16)
        cv2.imwrite(main_path_filename+"_obs.png", imgASuint16)
        #plt.imsave(main_path_filename+"_obs.jpg", ConvertGreyScale(observationimage, colorArray))
    
def saveDicomImage( root, purpose, case_id, dicomImage, main_mask, main_observation_mask, 
                    cropped_dicomImageWithROI, colorArray ):
    png_xray_images_directory   = root + "/png/"+purpose+"/xray_images/"
    png_main_obs_mask_directory = root + "/png/"+purpose+"/main_obs_masks/"
    png_breast_mask_directory   = root + "/png/"+purpose+"/breast_mask/"
    os.makedirs(png_xray_images_directory,   exist_ok=True)
    os.makedirs(png_main_obs_mask_directory, exist_ok=True)
    os.makedirs(png_breast_mask_directory,   exist_ok=True)
    
    
    png_xray_images_path_filename   = png_xray_images_directory  + "/" + case_id
    png_main_obs_mask_path_filename = png_main_obs_mask_directory  + "/" + case_id+"_alle_obs"
    png_breast_mask_path_filename   = png_breast_mask_directory + "/" + case_id+"_breast"    

    
    cv2.imwrite(png_xray_images_path_filename+".png", (65535.0*dicomImage + 0.5).astype(np.uint16))    
    #jpg_directory               = root + "/jpg/"+purpose+"/colored_xray_images/"
    #os.makedirs(jpg_directory,               exist_ok=True)
    #jpg_path_filename               = jpg_directory  + "/" + case_id+"_color_coded"
    #plt.imsave(jpg_path_filename, ConvertGreyScale(dicomImage, colorArray))

    if main_mask.max() > 0 :
        main_mask = ((main_mask*255.0+0.5)/main_mask.max()).astype(np.uint8)
    else:
        main_mask = main_mask.astype(np.uint8)                    
        dicomImage_issues.append( (case_id, "saveDicomImage: main_mask.max() == 0 ") )
    cv2.imwrite(png_breast_mask_path_filename+".png", main_mask )
                    
    if not main_observation_mask is None:        
#        if main_observation_mask.max() > 0 :
#            main_observation_mask = ((main_observation_mask*255.0+0.5)/main_observation_mask.max()).astype(np.uint8) 
#        else:    
        main_observation_mask = main_observation_mask.astype(np.uint8) 
#            dicomImage_issues.append( (case_id, "saveDicomImage: main_observation_mask.max() == 0 ") )
        cv2.imwrite(png_main_obs_mask_path_filename + ".png", main_observation_mask)
        
        
        xray_obs_borders_directory = root + "/observations/"+purpose+"/x_ray_with_obs_borders/"
        os.makedirs(xray_obs_borders_directory, exist_ok=True)
        xray_obs_borders_directory_path_filename = xray_obs_borders_directory + case_id + "_obs_borders"
        #cv2.imwrite(xray_obs_borders_directory_path_filename + ".png", (65535.0*cropped_dicomImageWithROI + 0.5).astype(np.uint16))
        xb = np.zeros( [cropped_dicomImageWithROI.shape[0],cropped_dicomImageWithROI.shape[1],3], np.uint8)
        xb[:,:,0] = xb[:,:,1] = xb[:,:,2] = (cropped_dicomImageWithROI*255.0).astype(np.uint8)
        cv2.imwrite(xray_obs_borders_directory_path_filename + ".jpg", xb)
        

In [154]:
def trimMaskRowWithPlot( main_mask, plotprofile=False):
    if plotprofile:
        profile_before = np.sum(main_mask,1)
        xcs = main_mask.cumsum(1)
        mid_cols = (main_mask.shape[1]-1)/2
        diffIdx_before = np.abs( np.add(xcs.argmax(1), np.argmax(xcs>0, 1))/2 - mid_cols )
    
    
    threshhold = 0.01
    main_mask, min_row, max_row = trimMaskRows( main_mask, threshhold )
    
    if plotprofile:
        mid_cols      = (main_mask.shape[1]-1)/2
        profile_after = np.sum(main_mask,1)
        xcs           = main_mask.cumsum(1)
        diffIdx_after = np.abs( np.add(xcs.argmax(1), np.argmax(xcs>0, 1))/2 - mid_cols )
        plotMaskProfile( profile_before, profile_after, diffIdx_before, diffIdx_after )
    
        print("started with nb rows: ", main_mask.shape[0], 
              "nb rows left: ", max_row-min_row+1, 
              "removed rows: ", main_mask.shape[0]-max_row+min_row-1, 
              "min row: ", min_row, "max row: ", max_row )
        
    return main_mask

xMinThick = 22 # Define minimum thickness
yMinThick = 22 # Define minimum thickness
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (xMinThick, yMinThick))
rotated_kernel = ndimage.rotate(kernel, -45.0, reshape=True)
#section = round(rotated_kernel.shape[1]/4+2)
#rotated_kernel = rotated_kernel[0:section, 0:section]
print( "kernel shape: ", kernel.shape, " rotated_kernel shape: ", rotated_kernel.shape )
print( "rotated_kernel:\n", rotated_kernel )
section = round(rotated_kernel.shape[1]/2)
rotated_kernel = rotated_kernel[:, 0:section]
print( "kernel shape: ", kernel.shape, " rotated_kernel shape: ", rotated_kernel.shape )
print( "rotated_kernel:\n", rotated_kernel )


In [155]:
def plot( case_id, dicomImage, cropped_cleanDicomImage, main_mask, main_observation_mask, colormap, colormap_grey ):

    x, y, angle = calcMasscenterOrientation( dicomImage*main_mask )
    seedPoint = (round(x),round(y))
    plotDicomWithSeedpoint(dicomImage, case_id, seedPoint, angle, 6, 50, colormap_grey)
    
    DoublePlot(case_id, dicomImage, "original image", main_mask, "main_mask", colormap_grey )

    plotHistogramsBeforeAndAfterMasking(dicomImage, cropped_cleanDicomImage, case_id)
        
    if not main_observation_mask is None :
        cropped_dicomImageWithROI = makeImageWithContour ( cropped_cleanDicomImage, main_observation_mask )
        DoublePlot(case_id, cropped_cleanDicomImage, "cropped_cleanDicomImage",  
                   cropped_dicomImageWithROI, " cropped_dicomImageWithROI", colormap )
        

In [156]:
def calcMask( image, flood_threshold=0.1, morph_iterations=15 ):
  
    # remove light parts of the border by setting a white border and then flood it with light 
    flood_mask = np.zeros(image.shape, dtype = np.uint8)
    corner_width = 250
    border_width =  46
    image = removeBorder(image, border_width, corner_width, 1)
    #simplePlot(image,"image_with border removed", cmap=plt.cm.gray)

    loDiff = 0.09
    upDiff = 1
    flags = 4 | cv2.FLOODFILL_MASK_ONLY | cv2.FLOODFILL_FIXED_RANGE
    maskSize = [image.shape[0]+2, image.shape[1]+2]
    flood_mask = np.zeros(maskSize, dtype= np.uint8)
    a = cv2.floodFill( image, flood_mask, (0,0), 1, loDiff, upDiff, flags )
    flood_mask = flood_mask[1:(maskSize[0]-1), 1:(maskSize[1]-1)]
    ix = flood_mask[:,:]==1
    image[ix] = 0     
    #simplePlot(flood_mask,"flooded border dicom", cmap=plt.cm.gray)


    #flood the breast area
    x, y, angle = calcMasscenterOrientation( image )
    seedPoint = (round(x),round(y))
    seedpoint_intensity = image[seedPoint[1], seedPoint[0]]
    loDiff = max(0.001, seedpoint_intensity - flood_threshold)
    upDiff = max(0.001,   1 - seedpoint_intensity - flood_threshold) 
    flags  = 4 | cv2.FLOODFILL_MASK_ONLY | cv2.FLOODFILL_FIXED_RANGE
    #print("seedPoint: ", seedPoint, " intesity: ", seedpoint_intensity, "loDiff: ", loDiff, " upDiff: ", upDiff )

    maskSize   = [image.shape[0]+2, image.shape[1]+2]
    flood_mask = np.zeros(maskSize, dtype= np.uint8)
    
    a = cv2.floodFill( image, flood_mask, seedPoint, 1, loDiff, upDiff, flags )
    #simplePlot(flood_mask,"first flood", cmap=plt.cm.gray)
    
    #close holes
    minThick = 25 # Define minimum thickness
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (minThick, minThick))    
    flood_mask = cv2.morphologyEx(flood_mask, cv2.MORPH_CLOSE, kernel) # "chop" half thickness from mask and dilate the eroded mask
    #simplePlot(flood_mask,"cv2.MORPH_CLOSE", cmap=plt.cm.gray)

    #remove strands in the top and bottom the image in particular
    xMinThick = 2 # Define minimum thickness
    yMinThick = 450 # Define minimum thickness
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (xMinThick, yMinThick))
    flood_mask = cv2.morphologyEx(flood_mask, cv2.MORPH_OPEN, kernel) 

    flood_mask = removeBorder(flood_mask, border_width, corner_width)
    
    #Assuming that the breast is orientede horizontally we employ at filter where the height is largest
    #The minimuze the impact on the border closes to the image border 
    flood_mask = cv2.blur(flood_mask,(100,250),borderType = cv2.BORDER_REPLICATE)
    #simplePlot(flood_mask,"cv2.blur", cmap=plt.cm.gray)

    flood_mask = flood_mask[1:(maskSize[0]-1), 1:(maskSize[1]-1)]

    #this should not be necessary
    cv2.threshold(flood_mask, 0, 255, cv2.THRESH_BINARY, dst=flood_mask);

    #should not be necessay
    flood_mask[flood_mask>0] = 1

    return flood_mask

In [157]:
def calcMainMask( dicomImage, plotprofile=False ):

    flood_threshold=0.03
    main_mask = calcMask( dicomImage, flood_threshold )
    
    main_mask = trimMaskRowWithPlot( main_mask, plotprofile)
    
    return main_mask

In [158]:
#linear fire
#colormapName = "linear_kryw_0_100_c71"
#colormap=cc.cm.linear_kryw_0_100_c71 #fire
#colorArray = np.asarray( cc.linear_kryw_0_100_c71 )

colormapName = "linear_kry_5_98_c75"
colormap     = cc.cm.linear_kry_5_98_c75 
colorArray   = np.asarray( cc.linear_kry_5_98_c75)

#linear green
#colormapName = "linear_green_5_95_c69"
#colormap = cc.cm.linear_green_5_95_c69
#colorArray = np.asarray( cc.linear_green_5_95_c69 )

colormap_grey = cc.cm.linear_grey_0_100_c0
greyArray     = np.asarray( cc.linear_grey_0_100_c0 )

#colormap=plt.cm.Greens   #we should use colorcet kgy or lienar green 0 til 1000, linear_kryw_0_100_c71
#colormapName = "inferno"
#colormap=plt.cm.inferno
#colormap = plt.cm.viridis   #1.813006, 2278

#colormap=plt.cm.Greys
#colormap=plt.cm.YlGn
#colormap=plt.cm.Greys_r
#colormap=plt.cm.plasma      #2.231636 2278
#colormap = plt.cm.GnBu
#colormap = plt.cm.viridis   #1.813006, 2278
#colorArray = np.asarray( colormap.colors )


In [159]:
#test patients

#colormap=plt.cm.cividis   #1.856948 2278
#print( " len(colormap.colors): ", len(colormap.colors))
#patient ID: P_00005  cropped_flood_mask.shape: (5056, 2761)
#patient ID: P_00001  cropped_flood_mask.shape: (4808, 3024)
#patient ID: P_00007  cropped_flood_mask.shape: (5356, 2986)
#patient ID: P_00014  cropped_flood_mask.shape: (4728, 3032)
#patient_ID = "P_01860"  #cropped_flood_mask.shape: (4321, 2056)


#patient_ID = "P_00005"
#patient_ID = "P_00209" #biggest dicom
#patient_ID = "P_00008" #har mange masker
#row = allData[allData.patient_id == patient_ID].iloc[0]
#row["image file path"]

In [160]:
sum(np.isin( allData.case_id, "Mass-Training_P_01103_RIGHT_MLO" ))

3

In [161]:
def mainLoop( allData ):
    ucs = allData.copy()

    #create dataframe with uniquecases
    data,unique_indices = np.unique(ucs.case_id,return_index=True)
    ucs = ucs.iloc[unique_indices]
    #drop the filenames
    ucs = ucs.drop(ucs.columns[ucs.columns.get_loc("subtlety")+1:],axis=1)


    dicomImage_issues = []
    observationImage_issues = []

    nb_processed = 0
    ucs.insert(1, "height", None )
    ucs.insert(2, "width", None )

    print("number of cases to process : ",ucs.shape[0])
    #print("ucs:\n",ucs)
    #your code here    

    for i in range(0,ucs.shape[0]):
        case_id = ucs.iloc[i].case_id
    
        #if round(nb_processed/10)*10==nb_processed :
        #print( "Current case : ", nb_processed, " number of cases : ", ucs.shape[0], "case_id: ", case_id,  )
        
        nb_processed += 1
        case_and_observations = allData[allData.case_id==case_id]

        row = case_and_observations.iloc[0]
        purpose = row.purpose
    
        dicomImage, issues = readDicomImages( row )
        if issues is not None and len(issues)>0: dicomImage_issues.append(issues)  
    
        main_mask = calcMainMask( dicomImage )
        x,y,w,h   = cropRect( main_mask )
    
        ucs.iloc[i,ucs.columns.get_loc("height")] = h
        ucs.iloc[i,ucs.columns.get_loc("width")] = w
    
        main_observation_mask = None
        for  j in range(0,case_and_observations.shape[0]):
            row = case_and_observations.iloc[j]

            roi_mask, observationimage, observationType, issues  = readObservationImages( row )
            if issues is not None and len(issues) > 0: observationImage_issues.append( issues )
        
            #add the ROI to the main_roi_image
            if not roi_mask is None:
                if ucs.iloc[i]["simple_pathology"] != row["simple_pathology"]: 
                    ix = ucs.columns.get_loc("simple_pathology")
                    ucs.iloc[i,ix] = ucs.iloc[i]["simple_pathology"]+";"+row["simple_pathology"]
                
                if main_observation_mask is None: main_observation_mask = np.zeros_like( main_mask, dtype=np.uint8 ) 
    
                if ( (roi_mask.shape[0] != main_observation_mask.shape[0]) or 
                     (roi_mask.shape[1] != main_observation_mask.shape[1]) ) :
                    #simplePlot(roi_mask, "roi_mask with wrong size", colormap)
                    msg = "Resizing roi_mask: " + str(roi_mask.shape) + " to main_observation_mask: " + str(main_observation_mask.shape) 
                    observationImage_issues.append( (row.obs_id, msg) )
                
                    roi_mask = cv2.resize( roi_mask, (main_observation_mask.shape[1], main_observation_mask.shape[0]) )
                '''
                BENING:    pixels in the mainmask are set to 128 where the 
                           observation mask i nonzero
                MALIGNANT: pixels in the mainmask are set to 255 where the 
                           observation mask i nonzero
                '''
                #simplePlot(roi_mask, "new roi_mask", colormap)
                if observationType==ObservationType.BENIGN:
                    assignment_mask = main_observation_mask < roi_mask
                    main_observation_mask[assignment_mask] = 128
                    #simplePlot(main_observation_mask, "BENING main_observation_mask", colormap_grey)
                if observationType==ObservationType.MALIGNANT:
                    assignment_mask = main_observation_mask < (roi_mask*255)
                    main_observation_mask[assignment_mask] = 255
                    #simplePlot(main_observation_mask, "MALIGNANT main_observation_mask", colormap)
                
                #crop the observation mask and save it
                roi_mask = crop( roi_mask, x,y,w,h  )

            saveObservation( CBIS_DDSM_Converted, purpose, row.obs_id, observationimage, roi_mask, colorArray )
            observationimage = roi_mask = None
        
        
        dicomImage_cropped = crop( dicomImage * main_mask, x,y,w,h )
        main_mask_cropped  = crop( main_mask, x,y,w,h )
        if not main_observation_mask is None:
            main_observation_mask     = crop( main_observation_mask, x,y,w,h )
        
            cropped_dicomImageWithROI = makeImageWithContour ( dicomImage_cropped, main_observation_mask )
        
            #treshold just in case the observation masks did not use the same value
            #cv2.threshold(main_observation_mask, 0, 255, cv2.THRESH_BINARY, dst=main_observation_mask);
        else:
            cropped_dicomImageWithROI = None
    
        #simplePlot(main_observation_mask, "main_observation_mask", colormap)
        #simplePlot(main_observation_mask, "main_observation_mask", colormap_grey)
    
        saveDicomImage( CBIS_DDSM_Converted, purpose, case_id, dicomImage_cropped, main_mask_cropped, 
                        main_observation_mask, cropped_dicomImageWithROI, colorArray )
        #plot( case_id, dicomImage, dicomImage_cropped, main_mask, main_observation_mask, colormap, colormap_grey )
        dicomImage = main_mask = main_mask_cropped = main_observation_mask = None

        if i> 0 and i-(int(i/50)*50) == 0 :
            percentage = int(np.around( i*100 / ucs.shape[0]))
            print( f"Current case: {row.case_id}  Processed cases: {percentage} %" )

    if len(observationImage_issues) > 0 or len(dicomImage_issues) > 0:
        print("observationImage_issues:\n", np.asarray(observationImage_issues) )
        print("dicomImage_issues:\n",       np.asarray(dicomImage_issues) )
    return ucs


In [162]:
#import line_profiler
#%reload_ext line_profiler

start = time.clock()
plt.rcParams['savefig.jpeg_quality'] = 95
 

"""
cases =  [
    #"Mass-Training_P_00419_LEFT_MLO",
    #"Mass-Training_P_00797_LEFT_CC",
    #"Calc-Training_P_00937_RIGHT_MLO",
    #"Mass-Test_P_00969_LEFT_CC"
    "Calc-Test_P_00038_LEFT_CC" #must be benign
    ]
ucs = allData.copy()
#display(ucs.head())
ucs = ucs[ np.isin( ucs.case_id, cases ) ]
#display(ucs.head())
ucs = mainLoop(ucs)
"""

ucs = mainLoop(allData)

"""
#%timeit mainLoop(ucs)
import line_profiler
lp = line_profiler.LineProfiler()
lp.add_function(calcMask)
lp.runctx('mainLoop(ucs)', locals=locals(), globals=globals())
lp.print_stats()
"""

#countColors()
ucs.to_csv(CBIS_DDSM_Converted + "/converted_data_with_dimensions.csv", sep=";")

print("time og main loop", time.clock() - start )


number of cases to process :  3101
Current case: Calc-Test_P_00353_LEFT_CC  Processed cases: 2 %
Current case: Calc-Test_P_00753_LEFT_MLO  Processed cases: 3 %
Current case: Calc-Test_P_01157_RIGHT_CC  Processed cases: 5 %
Current case: Calc-Test_P_01483_LEFT_CC  Processed cases: 6 %
Current case: Calc-Test_P_01868_LEFT_CC  Processed cases: 8 %
Current case: Calc-Training_P_00014_LEFT_MLO  Processed cases: 10 %
Current case: Calc-Training_P_00098_RIGHT_MLO  Processed cases: 11 %
Current case: Calc-Training_P_00232_RIGHT_CC  Processed cases: 13 %
Current case: Calc-Training_P_00307_LEFT_CC  Processed cases: 15 %
Current case: Calc-Training_P_00398_LEFT_MLO  Processed cases: 16 %
Current case: Calc-Training_P_00479_LEFT_MLO  Processed cases: 18 %
Current case: Calc-Training_P_00548_LEFT_MLO  Processed cases: 19 %
Current case: Calc-Training_P_00613_RIGHT_MLO  Processed cases: 21 %
Current case: Calc-Training_P_00680_LEFT_MLO  Processed cases: 23 %
Current case: Calc-Training_P_00767_LEFT